# Arbol

### Imports

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from joblib import dump, load

In [2]:
from sklearn.svm import SVC
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

In [3]:
import sklearn
print(sklearn.__version__)

1.0.1


In [4]:
from preprocessing import reemplazarNulls,reemplazarCategoricas,reemplazarFechas,targetBooleano,regularizar

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SimpleImputer from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(


[###] All Done                          


/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator Lasso from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
from utilities import score

### Codigo a correr

In [6]:
MAX_ITER = 10**6
MAX_JOBS = -1

**Run :** Entrenar Nuevamente o Cargar Entrenado

In [22]:
runLinearSimple = True
runLinear = False
runPoly = True
runRadial = True

**Save :** Guardar Modelo (pisa anterior)

In [23]:
saveLinear= True
saveLinearSimple= True
runPoly = True
runRadial = True

### Dataset

In [9]:
df_feat = pd.read_csv("datasets/train_features.csv", low_memory=False)
df_targ = targetBooleano( pd.read_csv("datasets/train_target.csv") )

### Preprocesamiento

In [10]:
reemplazarNulls(df_feat , inplace=True)
reemplazarCategoricas(df_feat , inplace=True)
reemplazarFechas(df_feat , inplace=True)
df_allfeat = regularizar(df_feat , drop=0)
regularizar(df_feat , inplace=True)

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SimpleImputer was fitted without feature names
  warnings.warn(
/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


,id,direccion_viento_tarde,direccion_viento_temprano,horas_de_sol,humedad_tarde,humedad_temprano,mm_lluvia_dia,nubosidad_tarde,presion_atmosferica_tarde,rafaga_viento_max_direccion,rafaga_viento_max_velocidad,velocidad_viendo_tarde
0,0.495844,0.736537,0.645996,-2.018030,1.277923,0.270523,7.914307,0.916539,1.269394,0.802215,0.589120,0.834079
1,0.183431,0.317434,-1.382279,-2.018030,-1.806334,-0.675093,0.417359,-1.288420,-2.406972,0.393094,1.544505,1.402051
2,-0.113001,-0.730326,-0.570969,-1.330311,1.326115,-1.147901,-0.281713,1.284033,-2.094694,-0.834272,1.250541,1.629239
3,-1.678190,1.574745,0.848823,-2.018030,1.952605,1.426276,0.320935,1.284033,0.020281,-1.447954,1.030067,1.402051
4,0.930508,-1.149430,-1.179451,-2.018030,-1.806334,-1.673244,-0.281713,-0.920927,-1.257220,-0.834272,0.736103,0.038918
...,...,...,...,...,...,...,...,...,...,...,...,...
102351,-0.264327,-0.101670,-0.570969,-0.298733,0.555051,-1.305504,-0.233501,0.549046,-0.490719,-0.834272,2.352908,2.083617
102352,-1.317418,0.946089,1.457306,-1.462565,0.651434,0.585729,0.345041,0.916539,0.318365,1.211337,0.589120,1.402051
102353,-1.594150,0.736537,1.660133,-2.018030,1.422498,-0.359888,-0.281713,0.916539,-0.064885,-1.447954,-0.366265,-1.892186
102354,0.293898,-1.149430,-0.773796,-1.779974,1.326115,-0.202285,-0.281713,1.284033,-2.052110,-0.629711,1.177050,0.606890


In [11]:
df_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102356 entries, 0 to 102355
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           102356 non-null  float64
 1   direccion_viento_tarde       102356 non-null  float64
 2   direccion_viento_temprano    102356 non-null  float64
 3   horas_de_sol                 102356 non-null  float64
 4   humedad_tarde                102356 non-null  float64
 5   humedad_temprano             102356 non-null  float64
 6   mm_lluvia_dia                102356 non-null  float64
 7   nubosidad_tarde              102356 non-null  float64
 8   presion_atmosferica_tarde    102356 non-null  float64
 9   rafaga_viento_max_direccion  102356 non-null  float64
 10  rafaga_viento_max_velocidad  102356 non-null  float64
 11  velocidad_viendo_tarde       102356 non-null  float64
dtypes: float64(12)
memory usage: 9.4 MB


## Entrenamiento

### Lineal

#### Simple

In [12]:
if (runLinearSimple):
    svm_simple_lineal = SVC(
        kernel='linear', C=0.1,
        max_iter=MAX_ITER, shrinking=False, verbose = 1,
    )
    svm_simple_lineal.fit( df_feat, df_targ )
    
    # Guardar
    if(saveLinearSimple):
        dump(svm_simple_lineal, 'models/SVM/svm_simple_lineal.sk') 
else:
    # Cargar
    svm_simple_lineal = load('models/SVM/svm_simple_lineal.sk')

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
pred = svm_simple_lineal.predict(df_feat)
score(df_targ, pred)

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


ACCURACY: 0.8385927546992848


#### Optimizando Hiperparametros

In [14]:
base_arrange = np.arange(0.01, 0.09, 0.03)
hipr_arrange = np.concatenate((base_arrange,base_arrange*10,base_arrange*100),axis=None)
hipr_arrange

array([0.01, 0.04, 0.07, 0.1 , 0.4 , 0.7 , 1.  , 4.  , 7.  ])

In [15]:
if (runLinear):
    svm_lineal = SVC(
        kernel='linear',
        max_iter=MAX_ITER, shrinking=False,
    )
    params = {'C': hipr_arrange}
    
    svm_lineal_grid = HalvingGridSearchCV(
        svm_lineal,
        params,
        scoring='accuracy',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=2,
    ).fit(df_feat, df_targ)
    
    # Guardar
    if(saveLinear):
        dump(svm_lineal_grid, 'models/SVM/svm_lineal_grid.sk') 
else:
    # Cargar
    svm_lineal_grid = load('models/SVM/svm_lineal_grid.sk')

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 3
min_resources_: 11372
max_resources_: 102356
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 9
n_resources: 11372
Fitting 5 folds for each of 9 candidates, totalling 45 fits
----------
iter: 1
n_candidates: 3
n_resources: 34116
Fitting 5 folds for each of 3 candidates, totalling 15 fits


/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/svm/_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


----------
iter: 2
n_candidates: 1
n_resources: 102348
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END .............................................C=0.04; total time=   2.6s
[CV] END ..............................C=0.06999999999999999; total time=   3.0s
[CV] END ..............................................C=0.4; total time=   4.3s
[CV] END ..............................................C=1.0; total time=   7.2s
[CV] END ..............................................C=4.0; total time=  19.9s
[CV] END ..............................................C=1.0; total time= 1.3min
[CV] END ................................C=6.999999999999999; total time= 5.2min
[CV] END .............................................C=0.04; total time=   2.7s
[CV] END ..............................C=0.06999999999999999; total time=   3.1s
[CV] END ..............................................C=0.4; total time=   4.0s
[CV] END ..............................................C=1.0; total time=   

In [16]:
print(f"Best score: {svm_lineal_grid.best_score_}")
print(f"Best params {svm_lineal_grid.best_params_}")

Best score: 0.8386942323663009
Best params {'C': 1.0}


In [17]:
svm_lineal = svm_lineal_grid.best_estimator_

In [18]:
pred = svm_lineal.predict(df_feat)
score(df_targ, pred)

ACCURACY: 0.8385927546992848


#### Otras Corridas

SVM Lineal , C=10, sin eliminar features poco significativos ni limitar iteraciones, t ~= 10hs \
El entranamiento fue removido debido a que tarda mucho y ni siquiera se esta optimizando el hiperparametro

In [19]:
svm_lineal_allfeat = load('models/SVM/svm_lineal_allfeat.sk')

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:324: UserWarning: Trying to unpickle estimator SVC from version 0.23.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [20]:
pred = svm_lineal_allfeat.predict(df_allfeat)
score(df_targ, pred)

/home/nicozu/orga/venv/lib/python3.9/site-packages/sklearn/base.py:438: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


ACCURACY: 0.8404294814177967


### Validado

In [ ]:
if (runPoly):
    svm_poly = SVC(
        kernel='poly',
        max_iter=MAX_ITER, shrinking=False,
    )
    params = {
        'C': hipr_arrange,
        'degree': hipr_arrange,
        'gamma': hipr_arrange,
        'coef0': hipr_arrange,
    }
    
    svm_poly_grid = HalvingGridSearchCV(
        svm_poly,
        params,
        scoring='accuracy',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1,
    ).fit(df_feat, df_targ)
    
    # Guardar
    if(savePoly):
        dump(svm_poly_grid, 'models/SVM/svm_poly_grid.sk') 
else:
    # Cargar
    svm_poly_grid = load('models/SVM/svm_poly_grid.sk')

In [ ]:
if (runRadial):
    svm_radial = SVC(
        kernel='rbf',
        max_iter=MAX_ITER, shrinking=False,
    )
    params = {
        'C': hipr_arrange,
        'gamma': hipr_arrange,
    }
    
    svm_radial_grid = HalvingGridSearchCV(
        svm_radial,
        params,
        scoring='accuracy',
        n_jobs=MAX_JOBS,
        return_train_score=True,
        verbose=1,
    ).fit(df_feat, df_targ)
    
    # Guardar
    if(saveRadial):
        dump(svm_radial_grid, 'models/SVM/svm_radial_grid.sk') 
else:
    # Cargar
    svm_radial_grid = load('models/SVM/svm_radial_grid.sk')

## Conclusion

**Lineal:** \
ACCURACY: 0.8283246707569659

**Grid:** \
Best score: 0.8396869725200906 \
Best params {'max_depth': 10, 'min_samples_leaf': 100}

**Random:** \

Exportamos el modelo de grid search porque sus hiperparametros son 'redondos'